In [1]:
import os, json, pandas as pd, traceback

In [2]:
# !pip install -U langchain-openai

In [3]:
#from langchain.chat_models import ChatOpenAI
# LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. 
# An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
from langchain_openai import ChatOpenAI


In [4]:
import os
from dotenv import load_dotenv

load_dotenv() # taking env variables from .env file. 

mykey=os.environ["OPENAI_API_KEY"]


In [5]:
mykey=os.environ["OPENAI_API_KEY"]

models=["gpt-3.5", "gpt-3.5-turbo", "gpt-4", "gpt-4-turbo"]  #  model choices 

# temperature   0 - more determinisic.   2 - more randomness 
llm=ChatOpenAI(openai_api_key=mykey, model_name="gpt-3.5-turbo", temperature=0.5)


In [6]:
from langchain.llms import OpenAI 
from langchain.prompts import PromptTemplate 
from langchain.chains import LLMChain 
from langchain.chains import SequentialChain 
from langchain.callbacks import get_openai_callback 
import PyPDF2


In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [8]:
TEMPLATE="""
You are an expert MCQ maker and an expert in the subject {subject}. It is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [13]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [14]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=False)

# g:\My Drive\backup jerry-200k(master copy)\AI\project_MCQ\env\lib\site-packages\langchain_core\_api\deprecation.py:139: 
# LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. 
# Use RunnableSequence, e.g., `prompt | llm` instead.

from langchain_core.runnables.base import RunnableSequence
quiz_chain2 = quiz_generation_prompt | llm  # , output_key="quiz", verbose=False


In [15]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [16]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [17]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=False)
review_chain2 = prompt=quiz_evaluation_prompt | llm 

In [18]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose = False)

generate_evaluate_chain2= quiz_chain2 | review_chain


In [19]:

subjects = [ "Algebra 2","Algebra","Geometry", "Machine Learning", "Data Science",   "Topology"]

NUMBER=5
SUBJECT=subjects[2] 
TONE="Difficult"

In [20]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain.invoke(
        {
        
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

In [21]:
#quiz=json.loads(response["quiz"])
qz = response.get("quiz", None)
qz

'\n{\n    "1": {\n        "mcq": "In a triangle ABC, if angle A = 60 degrees and angle B = 70 degrees, what is the measure of angle C?",\n        "options": {\n            "a": "50 degrees",\n            "b": "60 degrees",\n            "c": "70 degrees",\n            "d": "80 degrees"\n        },\n        "correct": "d"\n    },\n    "2": {\n        "mcq": "If a circle has a radius of 5 cm, what is the circumference of the circle?",\n        "options": {\n            "a": "10π cm",\n            "b": "15π cm",\n            "c": "20π cm",\n            "d": "25π cm"\n        },\n        "correct": "c"\n    },\n    "3": {\n        "mcq": "A rectangular prism has a length of 10 cm, width of 5 cm, and height of 3 cm. What is the total surface area of the prism?",\n        "options": {\n            "a": "120 cm^2",\n            "b": "150 cm^2",\n            "c": "180 cm^2",\n            "d": "210 cm^2"\n        },\n        "correct": "d"\n    },\n    "4": {\n        "mcq": "If a triangle has s

In [22]:
quiz=json.loads(qz)
quiz 

{'1': {'mcq': 'In a triangle ABC, if angle A = 60 degrees and angle B = 70 degrees, what is the measure of angle C?',
  'options': {'a': '50 degrees',
   'b': '60 degrees',
   'c': '70 degrees',
   'd': '80 degrees'},
  'correct': 'd'},
 '2': {'mcq': 'If a circle has a radius of 5 cm, what is the circumference of the circle?',
  'options': {'a': '10π cm', 'b': '15π cm', 'c': '20π cm', 'd': '25π cm'},
  'correct': 'c'},
 '3': {'mcq': 'A rectangular prism has a length of 10 cm, width of 5 cm, and height of 3 cm. What is the total surface area of the prism?',
  'options': {'a': '120 cm^2',
   'b': '150 cm^2',
   'c': '180 cm^2',
   'd': '210 cm^2'},
  'correct': 'd'},
 '4': {'mcq': 'If a triangle has sides of length 5 cm, 12 cm, and 13 cm, what type of triangle is it?',
  'options': {'a': 'Equilateral triangle',
   'b': 'Isosceles triangle',
   'c': 'Scalene triangle',
   'd': 'Right triangle'},
  'correct': 'd'},
 '5': {'mcq': 'If a square has a diagonal of length 10 cm, what is the area

In [35]:
from src.mcqgenerator.utils import read_file, get_table_data 
table_data=get_table_data(qz)
table_data

[{'MCQ': 'If a square has a diagonal of length 10 cm, what is the area of the square?',
  'Choices': 'a->25 cm^2 || b->50 cm^2 || c->75 cm^2 || d->100 cm^2',
  'correct': 'b'}]

In [34]:
json.loads(qz)

{'1': {'mcq': 'In a triangle ABC, if angle A = 60 degrees and angle B = 70 degrees, what is the measure of angle C?',
  'options': {'a': '50 degrees',
   'b': '60 degrees',
   'c': '70 degrees',
   'd': '80 degrees'},
  'correct': 'd'},
 '2': {'mcq': 'If a circle has a radius of 5 cm, what is the circumference of the circle?',
  'options': {'a': '10π cm', 'b': '15π cm', 'c': '20π cm', 'd': '25π cm'},
  'correct': 'c'},
 '3': {'mcq': 'A rectangular prism has a length of 10 cm, width of 5 cm, and height of 3 cm. What is the total surface area of the prism?',
  'options': {'a': '120 cm^2',
   'b': '150 cm^2',
   'c': '180 cm^2',
   'd': '210 cm^2'},
  'correct': 'd'},
 '4': {'mcq': 'If a triangle has sides of length 5 cm, 12 cm, and 13 cm, what type of triangle is it?',
  'options': {'a': 'Equilateral triangle',
   'b': 'Isosceles triangle',
   'c': 'Scalene triangle',
   'd': 'Right triangle'},
  'correct': 'd'},
 '5': {'mcq': 'If a square has a diagonal of length 10 cm, what is the area

In [27]:
quiz_table_data  = [] 
for key, value in quiz.items(): 
    print(f"in key: {key}" )
    mcq=value["mcq"]
    options = " || ".join( [f"{option}: {option_value}" for option, option_value in value["options"].items()] )
    correct = value["correct"]
    quiz_table_data.append({"Question #": key, "MCQ": mcq, "Choices": options, "Correct": correct })

quiz_table_data                       

in key: 1
in key: 2
in key: 3
in key: 4
in key: 5


[{'Question #': '1',
  'MCQ': 'In a triangle ABC, if angle A = 60 degrees and angle B = 70 degrees, what is the measure of angle C?',
  'Choices': 'a: 50 degrees || b: 60 degrees || c: 70 degrees || d: 80 degrees',
  'Correct': 'd'},
 {'Question #': '2',
  'MCQ': 'If a circle has a radius of 5 cm, what is the circumference of the circle?',
  'Choices': 'a: 10π cm || b: 15π cm || c: 20π cm || d: 25π cm',
  'Correct': 'c'},
 {'Question #': '3',
  'MCQ': 'A rectangular prism has a length of 10 cm, width of 5 cm, and height of 3 cm. What is the total surface area of the prism?',
  'Choices': 'a: 120 cm^2 || b: 150 cm^2 || c: 180 cm^2 || d: 210 cm^2',
  'Correct': 'd'},
 {'Question #': '4',
  'MCQ': 'If a triangle has sides of length 5 cm, 12 cm, and 13 cm, what type of triangle is it?',
  'Choices': 'a: Equilateral triangle || b: Isosceles triangle || c: Scalene triangle || d: Right triangle',
  'Correct': 'd'},
 {'Question #': '5',
  'MCQ': 'If a square has a diagonal of length 10 cm, wha

In [60]:
tt=pd.DataFrame(quiz_table_data)
tt

,Question #,MCQ,Choices,Correct
0,1,"In a right triangle, what is the name of the s...",a: Hypotenuse | b: Adjacent | c: Opposite | d:...,a
1,2,What is the sum of the interior angles of a pe...,a: 540 degrees | b: 360 degrees | c: 450 degre...,a
2,3,Which of the following is NOT a type of triang...,a: Equilateral | b: Isosceles | c: Scalene | d...,d
3,4,What is the formula for the area of a circle?,a: 2πr | b: πr^2 | c: πr | d: r^2,b
4,5,What is the name of a polygon with 8 sides?,a: Octagon | b: Hexagon | c: Decagon | d: Nonagon,a


In [23]:
type(qz)
qz.to_csv("aa.csv", index=False)

In [198]:
from datetime import datetime 

datetime.now().strftime('%Y%m%d%H%M%S')

'20240726161037'

In [199]:
LOG_FILE = f"{datetime.now().strftime('%Y%m%d%H%M%S')}.log"
print (LOG_FILE)

20240726161955.log


In [59]:
import os 
os.getcwd()

'g:\\My Drive\\backup jerry-200k(master copy)\\AI\\project_MCQ\\experiment'

In [200]:
log_path = os.path.join(os.getcwd(), "logs")
print(log_path)

g:\My Drive\backup jerry-200k(master copy)\AI\project_MCQ\experiment\logs


In [204]:
os.path.join(log_path, LOG_FILE)


'g:\\My Drive\\backup jerry-200k(master copy)\\AI\\project_MCQ\\experiment\\logs\\20240726161955.log'